# Detect Objects

- People + Trees
- Bus Stops + Signs

In [ ]:
import json

from os import listdir, makedirs, path
from PIL import Image as PImage, ImageDraw as PImageDraw

from utils.detect_utils import OBJECT_THRESHOLDS
from models.Dino import Dino
from models.Ocr import Ocr
from models.Yolo import Yolo

In [ ]:
OBJ_DIR = "./data/objs"
IMG_DIR = "./imgs"
TECHNIQUES = ["address", "address_coords", "heading", "station_coords"]

## People + Trees

In [ ]:
OBJS = "people_trees"
model = Dino()

for t in TECHNIQUES:
  makedirs(path.join(OBJ_DIR, t, OBJS), exist_ok=True)
  fnames = sorted(f for f in listdir(path.join(IMG_DIR, t)) if f.endswith("jpg"))

  for idx,f in enumerate(fnames):
    if idx>1024 and idx%16==0: print(idx, "/", len(fnames))

    img_file = path.join(IMG_DIR, t, f)
    obj_file = path.join(OBJ_DIR, t, OBJS, f.replace(".jpg", ".json"))

    if path.isfile(obj_file): continue

    img = PImage.open(img_file)

    objs = model.iou_objects(img, OBJECT_THRESHOLDS)

    with open(obj_file, "w") as ofp:
      json.dump({ "dino" : objs }, ofp)

## Bus Stops

In [ ]:
OBJS = "bus_stop"

OBJECT_THRESHOLDS_YOLO = {
  "bus_stop": 0.25,
  "bus_sign": 0.25,
}

model = Yolo("./models/bus_20260201_yolo.pt")

for t in TECHNIQUES:
  makedirs(path.join(OBJ_DIR, t, OBJS), exist_ok=True)
  fnames = sorted(f for f in listdir(path.join(IMG_DIR, t)) if f.endswith("jpg"))

  for idx,f in enumerate(fnames):
    if idx>0 and idx%16==0: print(idx, "/", len(fnames))

    img_file = path.join(IMG_DIR, t, f)
    obj_file = path.join(OBJ_DIR, t, OBJS, f.replace(".jpg", ".json"))

    if path.isfile(obj_file): continue

    img = PImage.open(img_file)

    objs = model.all_objects(img, OBJECT_THRESHOLDS_YOLO)

    with open(obj_file, "w") as ofp:
      json.dump({ "yolo" : objs }, ofp)

## Text

In [ ]:
OBJS = "text"

OCR_THRESHOLDS = {
  "area": 500,
  "score": 0.02
}

model = Ocr()

for t in TECHNIQUES[3:]:
  makedirs(path.join(OBJ_DIR, t, OBJS), exist_ok=True)
  fnames = sorted(f for f in listdir(path.join(IMG_DIR, t)) if f.endswith("jpg"))

  for idx,f in enumerate(fnames):
    if idx>0 and idx%16==0: print(idx, "/", len(fnames))

    img_file = path.join(IMG_DIR, t, f)
    obj_file = path.join(OBJ_DIR, t, OBJS, f.replace(".jpg", ".json"))

    if path.isfile(obj_file): continue

    words = model.top_words(img_file, OCR_THRESHOLDS["area"], OCR_THRESHOLDS["score"])
    lwords = [w.lower().strip().strip(".?!*,;:'\"") for w in words]

    with open(obj_file, "w") as ofp:
      json.dump({ "ocr" : lwords }, ofp)

## Combine JSONs

object labels -> imgids

In [ ]:
import json

from os import listdir, path

OBJ_DIR = "./data/objs"
TECHNIQUES = ["address", "address_coords", "heading", "station_coords"]

In [ ]:
# file id example: "address/1052"
label2id = {}
label2idboxes = {}
id2boxes = {}

for t in TECHNIQUES:
  obj_dirs = sorted([d for d in listdir(path.join(OBJ_DIR, t)) if path.isdir(path.join(OBJ_DIR, t, d))])
  for obj_dir in obj_dirs:
    if obj_dir == "text": continue
    obj_dir_path = path.join(OBJ_DIR, t, obj_dir)
    obj_files = sorted([f for f in listdir(obj_dir_path) if str(path.join(obj_dir_path, f)).endswith(".json")])
    for f in obj_files:
      fpath = path.join(obj_dir_path, f)
      with open(fpath, "r") as ifp:
        fid = t + "/" + f.replace('.json', '')
        obj_data = json.load(ifp)
        for model,boxes in obj_data.items():
          for b in boxes:
            b["box"] = [round(x, 4) for x in b["box"]]
            label2id[b["label"]] = label2id.get(b["label"], []) + [fid]
            id2boxes[fid] = id2boxes.get(fid, []) + [b]
            label2idboxes[b["label"]] = label2idboxes.get(b["label"], {})
            label2idboxes[b["label"]][fid] = label2idboxes[b["label"]].get(fid, []) + [b]


In [ ]:
with open("./data/objs/label2boxes.json", "w") as ofp:
  json.dump(label2idboxes, ofp)

## Combine OCR JSONs

word -> imgids

In [ ]:
import json
import re

from os import listdir, path

symbols = r"[!@#$%^&*().?!,;:0-9\-+\\/'\"]"
spaces = r"[ ]+"
repeats = r"([a-z])\1{N}"

OBJ_DIR = "./data/objs"
TECHNIQUES = ["address", "address_coords", "heading", "station_coords"]

In [ ]:
# file id example: "address/1052"
word2id = {}

for t in TECHNIQUES:
  obj_dir_path = path.join(OBJ_DIR, t, "text")
  obj_files = sorted([f for f in listdir(obj_dir_path) if str(path.join(obj_dir_path, f)).endswith(".json")])
  for f in obj_files:
    fpath = path.join(obj_dir_path, f)
    with open(fpath, "r") as ifp:
      fid = t + "/" + f.replace('.json', '')
      obj_data = json.load(ifp)
      for model,words in obj_data.items():
        words = list(set(words))
        for raw in words:
          if raw == "google": continue
          word = re.sub(spaces, " ", re.sub(symbols, "", raw)).strip()
          lw = len(word)
          most_repeats = re.match(repeats.replace("N", f"{lw-3},{lw-1}"), word)
          if (lw < 4) or (word == "google") or (most_repeats is not None): continue

          word2id[word] = word2id.get(word, []) + [fid]

In [ ]:
with open("./data/objs/word2images.json", "w") as ofp:
  json.dump(word2id, ofp)